1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수 있음
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 DB에 저장한다
4. 사용자의 질문에 대해서 벡터 DB에서 유사도가 높은 문서를 찾는다
5. 유사도가 높은 문서를 LLM 에 질문과 함께 전달

In [ ]:
%pip install docx2txt langchain-community langchain-text-splitters

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

loader = Docx2txtLoader("./tax.docx")
documents = loader.load_and_split(text_splitter)

print(len(documents))

187


In [1]:
%pip install -qU langchain-pinecone pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [4]:
%pip uninstall -y pinecone-notebooks

Found existing installation: pinecone-notebooks 0.1.1
Uninstalling pinecone-notebooks-0.1.1:
  Successfully uninstalled pinecone-notebooks-0.1.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

vector_store = PineconeVectorStore.from_documents(documents, embeddings, index_name=index_name)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 26 Apr 2025 02:47:24 GMT', 'Content-Type': 'application/json', 'Content-Length': '94', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '2781', 'x-pinecone-request-id': '3176221922650770893', 'x-envoy-upstream-service-time': '83', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Request size 3MB exceeds the maximum supported size of 2MB","details":[]}


In [6]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

retrieved_docs = vector_store.similarity_search(query)

print(retrieved_docs)

NameError: name 'vector_store' is not defined

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

[Question]
{query}
"""

response = llm.invoke(prompt)

print(response.content)

연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 소득세율과 공제액을 고려해야합니다. 2023년 기준으로 적용되는 소득세율은 다음과 같습니다:

1. 1,200만원 이하: 6%
2. 1,200만원 초과 ~ 4,600만원 이하: 15%
3. 4,600만원 초과 ~ 8,800만원 이하: 24%
4. 8,800만원 초과 ~ 1억 5천만원 이하: 35%

연봉 5천만원은 4,600만원 초과, 8,800만원 이하에 해당하므로 다음과 같이 계산합니다.

1. 1,200만원 이하: 1,200만원 × 6% = 72만원
2. 1,200만원 ~ 4,600만원 (3,400만원): 3,400만원 × 15% = 510만원
3. 4,600만원 ~ 5,000만원 (400만원): 400만원 × 24% = 96만원

총 소득세 계산:
- 72만원 + 510만원 + 96만원 = 678만원

따라서 연봉 5천만원인 직장인의 소득세는 얼추 678만원입니다. **단, 추가적인 세액공제나 세금감면(예: 인적공제, 보험료세액공제 등)을 고려해야 실제 납부할 세액은 달라질 수 있습니다.**


In [ ]:
%pip install langchain langchainhub

In [7]:
from langchain_openai import ChatOpenAI
from langchain import hub

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [8]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = vector_store.as_retriever(),
    chain_type_kwargs = {"prompt": prompt},
    return_source_documents=True
)

message = qa_chain.invoke({"query": query})

print(message)

NameError: name 'vector_store' is not defined

### 1
- 답변을 제대로 하지 못함
- 소득세율이 이미지로 표현되어 있어 테이블로 인식하지 못함
- 테이블로 인식하게 하려면 마크다운 문법으로 테이블을 표현해주어야 함

### 2
- 질문에는 '직장인' 이라는 표현을 사용했고, 소득세법 문서에는 '거주자' 라는 표현을 사용하고 있어 벡터 유사도 순위가 낮아짐
- LangChain Expression Language (LCEL) 로 효율 개선
